In [9]:
from scipy.stats import chi2

---
# Ejercicio 2
Para verificar que cierto dado no estaba trucado, se registraron $1000$ lanzamientos, resultando
que el número de veces que el dado arrojó el valor i $(i = 1, 2, 3, 4, 5, 6)$ fue, respectivamente, 158, 172, 164, 181, 160, 165. Aproximar el `p−valor` de la prueba: “el dado es honesto”

a) utilizando la prueba de **Pearson** con aproximación chi-cuadrada

## Parámetros especificados

Planteamos entonces como:

- *$H_0$: El dado es honesto*

Esto implica que como el dado tiene 6 valores posibles, la probabilidad de que salga un valor sea equiprobable con los otros, tal que:

$$
P(X=1) = P(X=2)= P(X=3) = P(X=4) = P(X=5) = P(X=6) = \frac{1}{6}
$$

Donde $X$ es la v.a que devuelve un número de un dado de "6 caras".


Luego las *frecuencias observadas* fueron $n=1000$:
- 158 -> 1
- 172 -> 2
- 164 -> 3
- 181 -> 4
- 160 -> 5
- 165 -> 6



Para aproximar el **p-valor** de esta muestra utilizando la prueba de Pearson con aproximación *Chi-Cuadrada* debemos tener en cuenta el estadístico
$$
T = \sum_{i=1}^n \frac{(N_i-n\times p_i)²}{n\times p_i}

$$
Tal que para nuestro caso tenemos:

$$
\begin{align*}
T &= \sum_{i=1}^6 \frac{(N_i-1000\times p_i)²}{1000\times p_i}\\[0.3cm]
  &=    \frac{(N_1-1000 \times p_1)²}{1000 \times p_1} +
        \frac{(N_2-1000 \times p_2)²}{1000 \times p_2} +
        \frac{(N_3-1000 \times p_3)²}{1000 \times p_3} +
        \frac{(N_4-1000 \times p_4)²}{1000 \times p_4} +
        \frac{(N_5-1000 \times p_5)²}{1000 \times p_5} +
        \frac{(N_6-1000 \times p_6)²}{1000 \times p_6}\\[0.3cm]

  &=    \frac{(158 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}} +
        \frac{(172 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}} +
        \frac{(164 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}} +
        \frac{(181 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}} +
        \frac{(160 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}} +
        \frac{(165 -1000 \times \frac{1}{6})²}{1000 \times \frac{1}{6}}\\[0.3cm]
  &=   0.451 + 0.170 + 0.043 + 1.232 + 0.267 + 0.017 = \boxed{2.18}
\end{align*}
$$

Ahora como hemos considerado 6 términos en el estadístico $T$, entonces debemos testear con una distribución $\chi_{6-1}^2$ tal que:
$$
p_{valor} = P(\chi_{5}^2 \geq 2.18) \in (0.75, 0.90)
$$


Ahora teniendo en cuenta que para un nivel de confianza del *95%*, como $p_{valor} > \alpha= 0.05$, la hipótesis nula *no* se rechaza pero no hay evidencias suficientes para hacerlo.

In [14]:
def Pearson_statistic(Nsamples:int, samples:list[int], probability_values:list[float]) -> float:
    statistic = 0
    for sample, prob_i in zip(samples, probability_values):
        statistic += (sample - Nsamples * prob_i) ** 2 / (Nsamples * prob_i)
    return statistic

def Pearson_test(Nsamples: int, samples:list[int], probability_values:list[float], alpha:float):
    T = Pearson_statistic(Nsamples=Nsamples, samples=samples, probability_values=probability_values)
    print(f"🧐 T estadístico:{round(T, 4)}")

    p_value = 1 - chi2.cdf(round(T, 4), df=len(samples)-1)

    print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

    if p_value > alpha:
        print("😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"🔴 Se rechaza Ho con una confianza del {100 * (100-alpha)}%")

In [15]:
#Valores constantes
DICE_SAMPLES = [158, 172, 164, 181, 160, 165] #1, 2, 3, 4, 5 y 6 respect
PROBABILITY_VALUES = [1/6] * 6
NSAMPLES = 1000

alpha = 1 - 0.95 # Test para el 95%

Pearson_test(Nsamples=NSAMPLES, samples=DICE_SAMPLES, probability_values=PROBABILITY_VALUES, alpha=alpha)

🧐 T estadístico:2.18
☝️ p-valor obtenido: 0.8237
😒 No hay evidencia suficiente para rechazar Ho


---
b) realizando una simulación.

A partir del estadístico obtenido $t_0 = 2.18$:

In [34]:
def p_value_estimation(Nsim:int, Nsamples:int, samples:list[int], probability_values:list[float],  t0:float):
    p_value = 0
    possible_frequencies = list(range(0, len(samples)))
    for _ in range(Nsim):
        #Generamos Nsamples muestras de las posibles frecuencias observadas,
        #cada una con probabilidad p_i
        X_samples = np.random.choice(a=possible_frequencies,  p=probability_values, size=Nsamples)

        N_i = np.bincount(X_samples, minlength=len(samples))
        
        T = Pearson_statistic(Nsamples=Nsamples, samples=N_i, probability_values=probability_values)

        if T > t0:
            p_value += 1

    return p_value / Nsim

In [35]:
NSIM = 10_000
T = 2.18
p_value = p_value_estimation(
            Nsim=NSIM,
            Nsamples=NSAMPLES,
            samples=DICE_SAMPLES,
            probability_values=PROBABILITY_VALUES,
            t0=T)

print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

☝️ p-valor obtenido: 0.8258
